# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,aasiaat,68.7098,-52.8699,30.54,80,56,7.36,GL
1,1,jamestown,42.0970,-79.2353,71.02,91,100,8.05,US
2,2,ocher,57.8904,54.7202,50.81,89,100,8.39,RU
3,3,reyes,-14.3167,-67.3833,63.66,69,100,4.32,BO
4,4,atherton,53.5237,-2.4935,57.74,78,100,3.00,GB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Plot cities 
cities_plot = city_data_df.hvplot.points("Lng", 
                                         "Lat", 
                                         geo = True, 
                                         size = "Humidity", 
                                         scale=1, 
                                         color="City", 
                                         alpha=0.5, 
                                         tiles="OSM", 
                                         frame_width = 700, 
                                         frame_height = 500)
cities_plot


/Users/rachelpuzycki/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df[(city_data_df['Humidity'] <=60) & (city_data_df["Max Temp"] >=30) & (city_data_df["Cloudiness"] <=25)]

# Drop any rows with null values
new_city_df.dropna()

# Display sample data
new_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
7,7,susuman,62.7833,148.1667,45.99,51,18,3.00,RU
24,24,san patricio,28.0170,-97.5169,96.17,46,13,20.98,US
46,46,brasilia,-15.7797,-47.9297,76.12,47,0,6.91,BR
52,52,lazaro cardenas,17.9583,-102.2000,86.65,48,0,11.48,MX
53,53,vicente guerrero,19.1194,-98.1708,83.59,33,3,14.65,MX


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
7,7,susuman,62.7833,148.1667,45.99,51,18,3.00,RU,
24,24,san patricio,28.0170,-97.5169,96.17,46,13,20.98,US,
46,46,brasilia,-15.7797,-47.9297,76.12,47,0,6.91,BR,
52,52,lazaro cardenas,17.9583,-102.2000,86.65,48,0,11.48,MX,
53,53,vicente guerrero,19.1194,-98.1708,83.59,33,3,14.65,MX,
...,...,...,...,...,...,...,...,...,...,...
519,519,kayenta,36.7278,-110.2546,71.22,17,9,27.27,US,
524,524,deming,32.2687,-107.7586,87.69,6,0,25.32,US,
545,545,arafat,21.3594,39.9664,88.57,32,0,5.53,SA,
556,556,kushk,33.2957,61.9522,69.89,44,2,12.21,AF,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
susuman - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
brasilia - nearest hotel: Hotel de Trânsito
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
vicente guerrero - nearest hotel: HOTEL SNAYLF
bethel - nearest hotel: Hampton Inn Danbury
mingora - nearest hotel: Chinar Hotel
asmar - nearest hotel: No hotel found
kas - nearest hotel: KEKOVA OTEL
crane - nearest hotel: No hotel found
burley - nearest hotel: Hampton Inn & Suites
yreka - nearest hotel: Best Western Minner's Inn
dar chioukh - nearest hotel: No hotel found
duba - nearest hotel: No hotel found
khairagarh - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
mil'kovo - nearest hotel: Дрозды
pont rouge - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
arar - nearest hotel: دار أوتيل Dar Otel
badiyah - nearest hotel: Al Areen Hotel Apartments
zinjibar - nearest hotel: No hotel found
kaka - nearest hotel: No hotel found
kuru

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
7,7,susuman,62.7833,148.1667,45.99,51,18,3.00,RU,No hotel found
24,24,san patricio,28.0170,-97.5169,96.17,46,13,20.98,US,No hotel found
46,46,brasilia,-15.7797,-47.9297,76.12,47,0,6.91,BR,Hotel de Trânsito
52,52,lazaro cardenas,17.9583,-102.2000,86.65,48,0,11.48,MX,Hotel Sol del Pacífico
53,53,vicente guerrero,19.1194,-98.1708,83.59,33,3,14.65,MX,HOTEL SNAYLF
...,...,...,...,...,...,...,...,...,...,...
519,519,kayenta,36.7278,-110.2546,71.22,17,9,27.27,US,Wetherill Inn
524,524,deming,32.2687,-107.7586,87.69,6,0,25.32,US,No hotel found
545,545,arafat,21.3594,39.9664,88.57,32,0,5.53,SA,ابراج فجر الايمان
556,556,kushk,33.2957,61.9522,69.89,44,2,12.21,AF,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700, 
    frame_height = 500,
    size = "Humidity",
    color = "Humidity",
    alpha=0.5,
    title = "Hotels in Cities with <60% Humidity, >30 Max Temp, and <25% Cloudiness"
)

# Display the map
map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)